In [2]:
decode(x::Float16) = (b=bitstring(x); (b[1], b[2:6], b[7:16]))
decode(x::Float32) = (b=bitstring(x); (b[1], b[2:9], b[10:32]))
decode(x::Float64) = (b=bitstring(x); (b[1], b[2:12], b[13:64]))

decode (generic function with 3 methods)

## Zadanie 1

In [3]:
[decode(Float16(1//3)), decode(Float32(1//3)), decode(Float64(1//3)), decode(Float64(Float16(1//3)))]

4-element Vector{Tuple{Char, String, String}}:
 ('0', "01101", "0101010101")
 ('0', "01111101", "01010101010101010101011")
 ('0', "01111111101", "0101010101010101010101010101010101010101010101010101")
 ('0', "01111111101", "0101010101000000000000000000000000000000000000000000")

## Zadanie 2

In [4]:
c = Float16[]
diffs = Float64[]
n = Float16(1)
while n < 1000000
    next_n = nextfloat(n)
    push!(diffs, Float64(next_n) - Float64(n))
    push!(c, n)
    n = next_n
end

In [ ]:
using Plots

plot(c, diffs, title="Distance between Float16 numbers", xlabel="Bigger number's value", ylabel="Distance", legend=false, size=(500, 400))

## Zadanie 3

```c
#include <stdio.h>
#include <gsl/gsl_ieee_utils.h>

int main() {
  float n = 0.001;
  while (n > 0.0) {
    n /= 2;
    gsl_ieee_printf_float(&n);
    printf("\n");
  }
  return 0;
}
```

``` 
 1.00000110001001001101111*2^-11
 1.00000110001001001101111*2^-12
 1.00000110001001001101111*2^-13
 1.00000110001001001101111*2^-14
...
 1.00000110001001001101111*2^-125
 1.00000110001001001101111*2^-126
 0.10000011000100100111000*2^-126
 0.01000001100010010011100*2^-126
 0.00100000110001001001110*2^-126
 0.00010000011000100100111*2^-126
 0.00001000001100010010100*2^-126
 0.00000100000110001001010*2^-126
 0.00000010000011000100101*2^-126
 0.00000001000001100010010*2^-126
 0.00000000100000110001001*2^-126
 0.00000000010000011000100*2^-126
 0.00000000001000001100010*2^-126
 0.00000000000100000110001*2^-126
 0.00000000000010000011000*2^-126
 0.00000000000001000001100*2^-126
 0.00000000000000100000110*2^-126
 0.00000000000000010000011*2^-126
 0.00000000000000001000010*2^-126
 0.00000000000000000100001*2^-126
 0.00000000000000000010000*2^-126
 0.00000000000000000001000*2^-126
 0.00000000000000000000100*2^-126
 0.00000000000000000000010*2^-126
 0.00000000000000000000001*2^-126
 0
```

## Zadanie 4

#### Podpunkt 1
Poniższy sposób obliczania potęgi liczby e jest niestabilny numerycznie, następuje catastrophic cancellation, wynik naszego szeregu znacząco różni się od faktycznej wartości.

In [41]:
maclaurin_series_e(n, x) = Float64(sum([Float16((x^i)/factorial(big(i))) for i in 0:n]))

maclaurin_series_e (generic function with 1 method)

In [42]:
n = 40
x = -5.5
series_result = maclaurin_series_e(n, x)
analytical_result = Base.MathConstants.e^x
series_result, analytical_result

(-0.00266265869140625, 0.004086771438464067)

#### Podpunkt 2
Przyjrzyjmy się bliżej catastrophic cancellation:

In [37]:
arr_float16 = [Float16((x^i)/factorial(big(i))) for i in 0:25]
arr_float64 = [Float64((x^i)/factorial(big(i))) for i in 0:25]
println(arr_float16)
println(arr_float64)

# Widzimy, że początkowo odejmując i dodając do siebie większe liczby, kumulujemy błąd,
# który w naszym końcowym, małym wyniku będzie istotny.

abs(sum(arr_float64[1:8]) - sum(arr_float16[1:8]))

Float16[1.0, -5.5, 15.125, -27.73, 38.12, -41.94, 38.44, -30.2, 20.77, -12.69, 6.98, -3.49, 1.6, -0.677, 0.2659, -0.0975, 0.0335, -0.01084, 0.003313, -0.000959, 0.0002637, -6.91e-5, 1.73e-5, -4.1e-6, 9.5e-7, -2.0e-7]
[1.0, -5.5, 15.125, -27.729166666666668, 38.127604166666664, -41.940364583333334, 38.44533420138889, -30.20704830109127, 20.76734570700025, -12.691155709833485, 6.980135640408417, -3.4900678202042084, 1.5996144175935956, -0.6767599459049827, 0.2658699787483861, -0.09748565887440823, 0.03351069523807783, -0.01084169551820165, 0.0033127402972282817, -0.0009589511386713448, 0.0002637115631346198, -6.906731415430518e-5, 1.7266828538576296e-5, -4.1290242157465054e-6, 9.462347161085741e-7, -2.0817163754388631e-7]


0.008858816964281147

#### Podpunkt 3
Żeby rozwiązać nasz problem wystarczy policzyć dodatnią potęgę, a następnie policzyć jej odwrotność.

In [44]:
n = 40
x = -5.5
1 / maclaurin_series_e(n, -x), Base.MathConstants.e^x

(0.00408371618172537, 0.004086771438464067)